In [4]:
from collections import defaultdict
import torch
import wfdb
import random


path  = "./data/mit-bih-arrhythmia-database-1.0.0"
dataset_divide = [0.6, 0.2, 0.2]

DS1 = [101, 106, 108, 109, 112, 114, 115, 116, 118, 119,
122, 124, 201, 203, 205, 207, 208, 209, 215, 220, 223, 230]
DS2 = [100, 103, 105, 111, 113, 117, 121, 123, 200, 202,
210, 212, 213, 214, 219, 221, 222, 228, 231, 232, 233, 234]
Total_DS = DS1 + DS2
random.shuffle(Total_DS)
DS_train = Total_DS[0:int(dataset_divide[0] * len(Total_DS))]
DS_eval = Total_DS[int(dataset_divide[0] * len(Total_DS)):int((dataset_divide[1] + dataset_divide[0]) * len(Total_DS))]
DS_test = Total_DS[int((dataset_divide[1] + dataset_divide[0]) * len(Total_DS)):]

# print(DS_train)


In [5]:

def mitdb_aryth_batchiter(batch_size, nums, path, use_last_batch=True):
    '''
    ###INPUT:
    batch_size: int, the batch_size of the iter
    nums: List, the ecg data sets to subsection
    path: str, the ecg data storage path
    use_last_batch: bool, if True, the final batch of an ecg data is not same as the batch_size; else, drop the final ecg data batch.

    ###FUNCTION:
    read a single ecg data, realize automatic segmentation
    a single heartbeat is consist of 258 
    
    ###OUTPUT:
    batch_x: 
    shape (batch_size x length x dimension, precisely equals to batch_size x 258 x 2)
    type  tensor.FloatTensor
    batch_y: shape (batch_size) 
    type  tensor.LongTensor
    
    '''
    # default dictionary for aryth classify
    default_dict_ecg = defaultdict(lambda: 4)
    dict_ecg_num =  {"N":0, "L":0, "R":0, "e":0, "j":0,      #"B":0,
                    "A":1, "a":1, "J":0, "S":1, 
                    "V":2, "E":2,                   #"r":2,
                    "F":3,                          #"n":3,
                    "/":4, "f":4, "Q":4, "|":4             #"?":4
                    }  
    for i in dict_ecg_num:
        default_dict_ecg[i] = dict_ecg_num[i]

    for num in nums:
        total_path = path + '/' + str(num)
        data = (wfdb.rdrecord(total_path, physical=False)).d_signal
        # data = std_standard(data)
        atr = wfdb.rdann(total_path, extension="atr")
        atr_sample = atr.sample
        atr_symbol = atr.symbol
        R_times = len(atr_symbol)
        batch_x = []
        batch_y = []
        count = 0
        for i in range(R_times - 2):
            if atr_sample[i+1] - atr_sample[i]>128:
                batch_x.append(torch.FloatTensor(data[atr_sample[i+1]-128:atr_sample[i+1]+128]).unsqueeze(dim=0)) # unsqueeze(dim=0): 增加第零维度
                batch_y.append(default_dict_ecg[atr_symbol[i+1]])
                count += 1
                if count == batch_size:
                    yield torch.cat(batch_x,dim=0), torch.LongTensor(batch_y)
                    count = 0
                    batch_x = []
                    batch_y = []
        if  count is not 0 and use_last_batch:
            yield torch.cat(batch_x,dim=0), torch.LongTensor(batch_y)

train_batch_iter = mitdb_aryth_batchiter(10, DS_train, path, use_last_batch=False)
print(train_batch_iter)

<generator object mitdb_aryth_batchiter at 0x7f207f7545d0>
